In [ ]:
!pip install spektral
!pip install Keras
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.8 MB/s eta 0:00:00


In [ ]:
import os

import math

import numpy as np
from numpy import dot
from numpy.linalg import norm

import pandas as pd

import networkx as nx

import matplotlib.pyplot as plt

import igraph
from igraph import Graph

import keras 
from keras.utils import to_categorical

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

import spektral
from spektral.layers import GatedGraphConv

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.regularizers import l2

from collections import Counter


In [ ]:
# the code below import the Cora dataset from Google Drive
import sys
runningOnColab = 'google.colab' in sys.modules
if runningOnColab:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd '/content/drive/MyDrive/Colab Notebooks'
!tar -xvf "cora.tgz"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
cora/
cora/README
cora/cora.cites
cora/cora.content


In [ ]:
# the code below obtain the data and the edge of the network
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
  print(files)
  for file in files:
    if '.content' in file:
      with open(os.path.join(root,file),'r') as f:
        all_data.extend(f.read().splitlines())
    elif 'cites' in file:
      with open(os.path.join(root,file),'r') as f:
        all_edges.extend(f.read().splitlines())

print(len(all_data))
print(len(all_data[0]))

['README', 'cora.cites', 'cora.content']
2708
2887


Sign the index for each categories or encode the categories, the order is decided by the order of aphabetical

In [ ]:
categories =  ['Reinforcement_Learning', 'Theory', 'Case_Based', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Rule_Learning']

# initialize the index
label_encoder = {}

# initialize the index
index = 0

for pos in sorted(categories):
  label_encoder[pos] = index
  index = index + 1

# display the indexed categories
print(label_encoder)

{'Case_Based': 0, 'Genetic_Algorithms': 1, 'Neural_Networks': 2, 'Probabilistic_Methods': 3, 'Reinforcement_Learning': 4, 'Rule_Learning': 5, 'Theory': 6}


In [ ]:
# obtain the needed dataset
# initialize the list for labels nodes and features of each data
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
  elements = data.split('\t')
  labels.append(label_encoder[elements[-1]])
  X.append(elements[1:-1])
  nodes.append(elements[0])
  element_to_ind[elements[0]]= i

X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)

#parse the edge
edge_list=[]
for edge in all_edges:
  e = edge.split('\t')
  edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {0, 1, 2, 3, 4, 5, 6}

Number of classes:  7


In [ ]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
print('Number of nodes', len(G.nodes), 'nodes')
print('Number of edges', len(G.edges), 'edges')

Number of nodes 2708 nodes
Number of edges 5278 edges


In [ ]:
nodes = list(G.nodes)
print(len(nodes))
list(G.neighbors(0))

2708


[258, 544, 8, 435, 14]

In [ ]:
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
df.head()

2708


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)

In [ ]:
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes 
df.head()

<ipython-input-12-2046024a68d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['node'] = list(range(len(df))) #rename nodes


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train = df.groupby('label', group_keys=False).apply(lambda x: x.sample(20))
G = nx.relabel_nodes(G, df['node'])

In [ ]:
def create_transition_matrix(g):
    vs = list(g.nodes)
    n = len(vs)
    adj = nx.adjacency_matrix(g)
    transition_matrix = adj/adj.sum(axis=1)

    return transition_matrix

In [ ]:
from networkx.algorithms.link_analysis.pagerank_alg import pagerank

the function below perform a random walk for a network

In [ ]:
def random_walk(g, num_steps, start_node, transition_matrix = None):
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)
  
  temp_nodes = range(transition_matrix.shape[0])
  
  # initialize the final arrived nodes
  V = start_node
  for index in range(num_steps):
    V = np.random.choice(temp_nodes, p = np.ravel(transition_matrix[V,]/np.sum(transition_matrix[V,]), order = 'C'))

  return V


In [ ]:
seeds_dict = {predicted:list(train[train['label'] == predicted]['node']) for predicted in range(7)}

def random_walk_with_teleportation(g, num_steps, start_node,tp,predicted, transition_matrix = None):
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)
  temp_seeds = seeds_dict[predicted]
  temp_nodes = range(transition_matrix.shape[0])
  
  # initialize the final arrived nodes
  V = start_node
  for index in range(num_steps):
    walk = np.random.uniform(low=0.0, high=1.0, size=None)
    if walk < tp:
      V = np.random.choice(temp_seeds)
    else:
      V = np.random.choice(temp_nodes, p = np.ravel(transition_matrix[V,]/np.sum(transition_matrix[V,]), order = 'C'))

  return V


(a) teleportation takes the random walker to one of the seed documents of that class (with a uniform probability of 1/20 per seed document).

In [ ]:
#pagerank. NO teleportation, NO tfidf. 
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]


for train_node,predicted in zip(train['node'],train['label']):
  print (train_node,predicted)
  for i in range(num_samples):
    start_point = train_node
    end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
    visiting_freq_label[end_node][predicted] += 1
    visiting_freq[end_node] +=1

268 0
1061 0
1064 0
771 0
1415 0
645 0
870 0
2441 0
943 0
979 0
1060 0
849 0
1142 0
897 0
546 0
2368 0
413 0
1460 0
2312 0
1855 0
1398 1
378 1
2036 1
1690 1
830 1
2423 1
976 1
850 1
974 1
1814 1
2090 1
1429 1
734 1
1203 1
640 1
810 1
138 1
2342 1
1149 1
975 1
1545 2
1886 2
790 2
1558 2
877 2
329 2
320 2
962 2
15 2
814 2
2459 2
374 2
1911 2
2418 2
223 2
1370 2
231 2
1972 2
10 2
130 2
467 3
2474 3
1022 3
2150 3
1197 3
2343 3
1776 3
1260 3
643 3
602 3
2236 3
165 3
1934 3
792 3
1742 3
622 3
114 3
75 3
41 3
2201 3
1358 4
1744 4
387 4
3 4
2440 4
2450 4
209 4
2383 4
360 4
880 4
1597 4
1299 4
259 4
2225 4
266 4
727 4
2469 4
1145 4
72 4
2136 4
1527 5
1238 5
951 5
948 5
270 5
1937 5
1012 5
153 5
1906 5
949 5
938 5
1220 5
2384 5
1764 5
1974 5
1140 5
1363 5
1099 5
827 5
1 5
947 6
1417 6
716 6
2046 6
542 6
1034 6
1746 6
1312 6
479 6
1748 6
1202 6
485 6
1659 6
2173 6
250 6
2153 6
1420 6
111 6
1606 6
2142 6


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
  
print('The Pagerank Random Walk Without Teleporation (i.e. tele_prob = 0) and Without TFIDF')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

The Pagerank Random Walk Without Teleporation (i.e. tele_prob = 0) and Without TFIDF
unvisited =  2
              precision    recall  f1-score   support

           0       0.65      0.71      0.68       285
           1       0.76      0.90      0.83       406
           2       0.83      0.52      0.64       726
           3       0.73      0.70      0.72       379
           4       0.47      0.56      0.51       214
           5       0.30      0.83      0.44       131
           6       0.60      0.45      0.51       344

    accuracy                           0.64      2485
   macro avg       0.62      0.67      0.62      2485
weighted avg       0.69      0.64      0.65      2485



0.6434607645875251

Teleportation probability is equal to 0.1

In [ ]:
# pagerank with telportation, without tfidf 
# repeat above expeiment but this time use the teleportation random walk 
# get transition matrix
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

# set the teleportation probability (here is equal to 0)
tele_prob = 0.1

for train_node,predicted in zip(train['node'],train['label']):
  print (train_node,predicted)
  for i in range(num_samples):
    start_point = train_node
    end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tele_prob, predicted, transition_matrix)        
    visiting_freq_label[end_node][predicted] += 1
    visiting_freq[end_node] +=1

268 0
1061 0
1064 0
771 0
1415 0
645 0
870 0
2441 0
943 0
979 0
1060 0
849 0
1142 0
897 0
546 0
2368 0
413 0
1460 0
2312 0
1855 0
1398 1
378 1
2036 1
1690 1
830 1
2423 1
976 1
850 1
974 1
1814 1
2090 1
1429 1
734 1
1203 1
640 1
810 1
138 1
2342 1
1149 1
975 1
1545 2
1886 2
790 2
1558 2
877 2
329 2
320 2
962 2
15 2
814 2
2459 2
374 2
1911 2
2418 2
223 2
1370 2
231 2
1972 2
10 2
130 2
467 3
2474 3
1022 3
2150 3
1197 3
2343 3
1776 3
1260 3
643 3
602 3
2236 3
165 3
1934 3
792 3
1742 3
622 3
114 3
75 3
41 3
2201 3
1358 4
1744 4
387 4
3 4
2440 4
2450 4
209 4
2383 4
360 4
880 4
1597 4
1299 4
259 4
2225 4
266 4
727 4
2469 4
1145 4
72 4
2136 4
1527 5
1238 5
951 5
948 5
270 5
1937 5
1012 5
153 5
1906 5
949 5
938 5
1220 5
2384 5
1764 5
1974 5
1140 5
1363 5
1099 5
827 5
1 5
947 6
1417 6
716 6
2046 6
542 6
1034 6
1746 6
1312 6
479 6
1748 6
1202 6
485 6
1659 6
2173 6
250 6
2153 6
1420 6
111 6
1606 6
2142 6


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1

print('The Pagerank Random Walk With Teleporation (tele_prob = 0.1) and Without TFIDF')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

The Pagerank Random Walk With Teleporation (tele_prob = 0.1) and Without TFIDF
unvisited =  15
              precision    recall  f1-score   support

           0       0.73      0.72      0.73       285
           1       0.84      0.90      0.87       406
           2       0.88      0.56      0.68       726
           3       0.67      0.80      0.73       379
           4       0.57      0.84      0.68       214
           5       0.53      0.86      0.65       131
           6       0.68      0.65      0.66       344

    accuracy                           0.72      2485
   macro avg       0.70      0.76      0.71      2485
weighted avg       0.75      0.72      0.72      2485



0.7215291750503018

Teleportation probability is equal to 0.2

In [ ]:
# pagerank with telportation, without tfidf 
# repeat above expeiment but this time use the teleportation random walk 
# get transition matrix
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

# set the teleportation probability (here is equal to 0)
tele_prob = 0.1

for train_node,predicted in zip(train['node'],train['label']):
  print (train_node,predicted)
  for i in range(num_samples):
    start_point = train_node
    end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tele_prob, predicted, transition_matrix)        
    visiting_freq_label[end_node][predicted] += 1
    visiting_freq[end_node] +=1

268 0
1061 0
1064 0
771 0
1415 0
645 0
870 0
2441 0
943 0
979 0
1060 0
849 0
1142 0
897 0
546 0
2368 0
413 0
1460 0
2312 0
1855 0
1398 1
378 1
2036 1
1690 1
830 1
2423 1
976 1
850 1
974 1
1814 1
2090 1
1429 1
734 1
1203 1
640 1
810 1
138 1
2342 1
1149 1
975 1
1545 2
1886 2
790 2
1558 2
877 2
329 2
320 2
962 2
15 2
814 2
2459 2
374 2
1911 2
2418 2
223 2
1370 2
231 2
1972 2
10 2
130 2
467 3
2474 3
1022 3
2150 3
1197 3
2343 3
1776 3
1260 3
643 3
602 3
2236 3
165 3
1934 3
792 3
1742 3
622 3
114 3
75 3
41 3
2201 3
1358 4
1744 4
387 4
3 4
2440 4
2450 4
209 4
2383 4
360 4
880 4
1597 4
1299 4
259 4
2225 4
266 4
727 4
2469 4
1145 4
72 4
2136 4
1527 5
1238 5
951 5
948 5
270 5
1937 5
1012 5
153 5
1906 5
949 5
938 5
1220 5
2384 5
1764 5
1974 5
1140 5
1363 5
1099 5
827 5
1 5
947 6
1417 6
716 6
2046 6
542 6
1034 6
1746 6
1312 6
479 6
1748 6
1202 6
485 6
1659 6
2173 6
250 6
2153 6
1420 6
111 6
1606 6
2142 6


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1

print('The Pagerank Random Walk with Teleporation (tele_prob = 0.2) and Without TFIDF')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

The Pagerank Random Walk with Teleporation (tele_prob = 0.2) and Without TFIDF
unvisited =  14
              precision    recall  f1-score   support

           0       0.72      0.74      0.73       285
           1       0.86      0.89      0.87       406
           2       0.86      0.55      0.67       726
           3       0.66      0.79      0.72       379
           4       0.54      0.83      0.66       214
           5       0.53      0.84      0.65       131
           6       0.66      0.62      0.64       344

    accuracy                           0.71      2485
   macro avg       0.69      0.75      0.71      2485
weighted avg       0.74      0.71      0.71      2485



0.7122736418511066

(b) the probability of transitioning to neighbors is not uniform among the neighbors. Rather, it is proportional to the cosine similarity between the text features of the current node and the next neighboring node.

In [ ]:
vs = list(G.nodes)
n = len(vs)
adj = nx.adjacency_matrix(G)
transition = np.zeros((len(G.nodes), len(G.nodes)))

for n1 in nodes:
  for n2 in nodes:
    # if there is an edge between n1 and n2:
    if G.has_edge(n1, n2):
      
      # Obtain the feature of n1 and n2 from TFIDF
      feature_1 = list(df['features'])[n1]
      feature_2 = list(df['features'])[n2]

      # cos_sim = compute cosine similarity between features of n1 and n2 
      # cosine similarity = (A . B) / (||A|| * ||B||)
      cos_sim = np.dot(feature_1, feature_2)/(norm(feature_1)*norm(feature_2))

      # transition[n1,n2] = np.exp(cos_sim) #neumerator of softmax. 
      # softmax is used to calculate the probability
      transition[n1,n2] = np.exp(cos_sim)
  
#divide the values in transition by denominator of softmax. 
transition = transition/transition.sum(axis=1, keepdims=True) 
      


In [ ]:
# pagerank without telportation, with tfidf 
# perfrom pagerank using our tf_idf based transition matrix
transition_matrix = transition

#use randon walk without teleporation 
# get transition matrix
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

for train_node,predicted in zip(train['node'],train['label']):
  print (train_node,predicted)
  for i in range(num_samples):
    start_point = train_node
    end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
    visiting_freq_label[end_node][predicted] += 1
    visiting_freq[end_node] +=1


268 0
1061 0
1064 0
771 0
1415 0
645 0
870 0
2441 0
943 0
979 0
1060 0
849 0
1142 0
897 0
546 0
2368 0
413 0
1460 0
2312 0
1855 0
1398 1
378 1
2036 1
1690 1
830 1
2423 1
976 1
850 1
974 1
1814 1
2090 1
1429 1
734 1
1203 1
640 1
810 1
138 1
2342 1
1149 1
975 1
1545 2
1886 2
790 2
1558 2
877 2
329 2
320 2
962 2
15 2
814 2
2459 2
374 2
1911 2
2418 2
223 2
1370 2
231 2
1972 2
10 2
130 2
467 3
2474 3
1022 3
2150 3
1197 3
2343 3
1776 3
1260 3
643 3
602 3
2236 3
165 3
1934 3
792 3
1742 3
622 3
114 3
75 3
41 3
2201 3
1358 4
1744 4
387 4
3 4
2440 4
2450 4
209 4
2383 4
360 4
880 4
1597 4
1299 4
259 4
2225 4
266 4
727 4
2469 4
1145 4
72 4
2136 4
1527 5
1238 5
951 5
948 5
270 5
1937 5
1012 5
153 5
1906 5
949 5
938 5
1220 5
2384 5
1764 5
1974 5
1140 5
1363 5
1099 5
827 5
1 5
947 6
1417 6
716 6
2046 6
542 6
1034 6
1746 6
1312 6
479 6
1748 6
1202 6
485 6
1659 6
2173 6
250 6
2153 6
1420 6
111 6
1606 6
2142 6


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
  
print('The Pagerank Random Walk Without Teleporation (i.e. tele_prob = 0) and With TFIDF')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

The Pagerank Random Walk Without Teleporation (i.e. tele_prob = 0) and With TFIDF
unvisited =  1
              precision    recall  f1-score   support

           0       0.66      0.72      0.69       285
           1       0.77      0.91      0.83       406
           2       0.84      0.52      0.65       726
           3       0.75      0.69      0.72       379
           4       0.48      0.58      0.53       214
           5       0.28      0.76      0.40       131
           6       0.57      0.47      0.51       344

    accuracy                           0.64      2485
   macro avg       0.62      0.66      0.62      2485
weighted avg       0.70      0.64      0.65      2485



0.6426559356136821

Teleportation probability is equal to 0.1

In [ ]:
# pagerank with telportation, without tfidf 
# repeat above expeiment but this time use the teleportation random walk 
# get transition matrix
transition_matrix = transition

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

# set the teleportation probability (here is equal to 0)
tele_prob = 0.1

for train_node,predicted in zip(train['node'],train['label']):
  print (train_node,predicted)
  for i in range(num_samples):
    start_point = train_node
    end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tele_prob, predicted, transition_matrix)        
    visiting_freq_label[end_node][predicted] += 1
    visiting_freq[end_node] +=1

268 0
1061 0
1064 0
771 0
1415 0
645 0
870 0
2441 0
943 0
979 0
1060 0
849 0
1142 0
897 0
546 0
2368 0
413 0
1460 0
2312 0
1855 0
1398 1
378 1
2036 1
1690 1
830 1
2423 1
976 1
850 1
974 1
1814 1
2090 1
1429 1
734 1
1203 1
640 1
810 1
138 1
2342 1
1149 1
975 1
1545 2
1886 2
790 2
1558 2
877 2
329 2
320 2
962 2
15 2
814 2
2459 2
374 2
1911 2
2418 2
223 2
1370 2
231 2
1972 2
10 2
130 2
467 3
2474 3
1022 3
2150 3
1197 3
2343 3
1776 3
1260 3
643 3
602 3
2236 3
165 3
1934 3
792 3
1742 3
622 3
114 3
75 3
41 3
2201 3
1358 4
1744 4
387 4
3 4
2440 4
2450 4
209 4
2383 4
360 4
880 4
1597 4
1299 4
259 4
2225 4
266 4
727 4
2469 4
1145 4
72 4
2136 4
1527 5
1238 5
951 5
948 5
270 5
1937 5
1012 5
153 5
1906 5
949 5
938 5
1220 5
2384 5
1764 5
1974 5
1140 5
1363 5
1099 5
827 5
1 5
947 6
1417 6
716 6
2046 6
542 6
1034 6
1746 6
1312 6
479 6
1748 6
1202 6
485 6
1659 6
2173 6
250 6
2153 6
1420 6
111 6
1606 6
2142 6


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1

print('The Pagerank Random Walk With Teleporation (tele_prob = 0.1) and With TFIDF')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

The Pagerank Random Walk With Teleporation (tele_prob = 0.1) and With TFIDF
unvisited =  9
              precision    recall  f1-score   support

           0       0.76      0.71      0.73       285
           1       0.83      0.93      0.88       406
           2       0.88      0.60      0.71       726
           3       0.73      0.80      0.76       379
           4       0.60      0.83      0.69       214
           5       0.46      0.91      0.61       131
           6       0.72      0.64      0.68       344

    accuracy                           0.74      2485
   macro avg       0.71      0.77      0.72      2485
weighted avg       0.77      0.74      0.74      2485



0.7360160965794769

Teleportation probability is equal to 0.2

In [ ]:
# pagerank with telportation, without tfidf 
# repeat above expeiment but this time use the teleportation random walk 
# get transition matrix
transition_matrix = transition

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

# set the teleportation probability (here is equal to 0)
tele_prob = 0.2

for train_node,predicted in zip(train['node'],train['label']):
  print (train_node,predicted)
  for i in range(num_samples):
    start_point = train_node
    end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tele_prob, predicted, transition_matrix)        
    visiting_freq_label[end_node][predicted] += 1
    visiting_freq[end_node] +=1

268 0
1061 0
1064 0
771 0
1415 0
645 0
870 0
2441 0
943 0
979 0
1060 0
849 0
1142 0
897 0
546 0
2368 0
413 0
1460 0
2312 0
1855 0
1398 1
378 1
2036 1
1690 1
830 1
2423 1
976 1
850 1
974 1
1814 1
2090 1
1429 1
734 1
1203 1
640 1
810 1
138 1
2342 1
1149 1
975 1
1545 2
1886 2
790 2
1558 2
877 2
329 2
320 2
962 2
15 2
814 2
2459 2
374 2
1911 2
2418 2
223 2
1370 2
231 2
1972 2
10 2
130 2
467 3
2474 3
1022 3
2150 3
1197 3
2343 3
1776 3
1260 3
643 3
602 3
2236 3
165 3
1934 3
792 3
1742 3
622 3
114 3
75 3
41 3
2201 3
1358 4
1744 4
387 4
3 4
2440 4
2450 4
209 4
2383 4
360 4
880 4
1597 4
1299 4
259 4
2225 4
266 4
727 4
2469 4
1145 4
72 4
2136 4
1527 5
1238 5
951 5
948 5
270 5
1937 5
1012 5
153 5
1906 5
949 5
938 5
1220 5
2384 5
1764 5
1974 5
1140 5
1363 5
1099 5
827 5
1 5
947 6
1417 6
716 6
2046 6
542 6
1034 6
1746 6
1312 6
479 6
1748 6
1202 6
485 6
1659 6
2173 6
250 6
2153 6
1420 6
111 6
1606 6
2142 6


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1

print('The Pagerank Random Walk With Teleporation (tele_prob = 0.1) and With TFIDF')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

The Pagerank Random Walk With Teleporation (tele_prob = 0.1) and With TFIDF
unvisited =  41
              precision    recall  f1-score   support

           0       0.69      0.72      0.70       285
           1       0.82      0.90      0.86       406
           2       0.86      0.59      0.70       726
           3       0.70      0.79      0.74       379
           4       0.60      0.80      0.68       214
           5       0.50      0.87      0.64       131
           6       0.71      0.62      0.67       344

    accuracy                           0.72      2485
   macro avg       0.70      0.76      0.71      2485
weighted avg       0.75      0.72      0.72      2485



0.723943661971831